Modul zur Konvertierung von HTML nach PDF installieren

In [5]:
!pip install xhtml2pdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.0/447.0 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 3.4 MB/s eta 0:00:00
  Created wheel for svglib: filename=svglib-1.5.1-py3-none-any.whl size=30906 sha256=6f565cd487ee08d2f0dece509bef044c34ca355cc8b3c58b880e77411e80d5af
  Stored 

JSON Charakter Datei laden

In [6]:
import json
import re
import textwrap
from xhtml2pdf import pisa

TAG_RE = re.compile(r'<[^>]+>')

json_file="/content/drive/MyDrive/Colab Notebooks/fvtt-Actor-hesindian-schlangentreu-pCXAwjkUPBP5X2uz.json"

with open(json_file, 'r') as f:
    data = json.load(f)

#print(data)

In [15]:
class DSAEquipmentData():
  name = None
  points = None
  description = None

  def __init__(self, name, points):
    self.name = name
    self.points = points
    self.description = "n/a"

class DSAChar():
  name = None
  details = None
  life_points = None
  astralenergy = None
  fate_points = None
  experience_total = None
  experience_spent = None
  characteristics = None
  char_items = None

  def __init__(self, name):
    self.name = name
    self.details = {}
    self.life_points = 0
    self.astralenergy = 0
    self.fate_points = 0
    self.experience_total = 0
    self.experience_spent = 0
    self.characteristics = {}
    self.char_items = {}


In [29]:
dsa_char = DSAChar(data["name"])

dsa_char.life_points = data["system"]["status"]["wounds"]["value"]
dsa_char.astralenergy = data["system"]["status"]["astralenergy"]["value"]
dsa_char.fate_points = data["system"]["status"]["fatePoints"]["value"]

dsa_char.experience_total = data["system"]["details"]["experience"]["total"]
dsa_char.experience_spent = data["system"]["details"]["experience"]["spent"]

for key,value in data["system"]["details"].items():
    if "value" in value:
        dsa_char.details[key] = '<br/>\n'.join(textwrap.wrap(TAG_RE.sub('', value["value"]), width=80))
        #dsa_char.details[key] = textwrap.shorten(TAG_RE.sub('', value["value"]), width=40)
        if len(dsa_char.details[key]) < 1:
            dsa_char.details[key] = "n/a"

for key,value in data["system"]["characteristics"].items():
    dsa_char.characteristics[value["label"]] = value["initial"] + value["advances"]

for item in data["items"]:
    item_name = item["name"]
    item_type = item["type"]
    if item_type not in dsa_char.char_items:
        dsa_char.char_items[item_type] = {}

    if "quantity" in item["system"]:
        dsa_char.char_items[item_type][item_name] = DSAEquipmentData(item_name, item["system"]["quantity"]["value"])
    elif "talentValue" in item["system"]:
        dsa_char.char_items[item_type][item_name] = DSAEquipmentData(item_name, item["system"]["talentValue"]["value"])
    elif "maxRank" in item["system"]:
        dsa_char.char_items[item_type][item_name] = DSAEquipmentData(item_name, item["system"]["maxRank"]["value"])
    elif "APValue" in item["system"]:
        dsa_char.char_items[item_type][item_name] = DSAEquipmentData(item_name, item["system"]["APValue"]["value"])
    else:
        print("Fehlt: " + item["name"])
        continue
    dsa_char.char_items[item_type][item_name].description = '<br/>\n'.join(textwrap.wrap(TAG_RE.sub('', item["system"]["description"]["value"]), width=80))
    if len(dsa_char.char_items[item_type][item_name].description) < 1:
        dsa_char.char_items[item_type][item_name].description = "n/a"

#print(dsa_char.name)
#print(dsa_char.details)
#print(dsa_char.characteristics)
#print(dsa_char.char_items)

In [51]:
# Create HTML for visualization
html_content = '''<!DOCTYPE html>
        <html>
        <head>
            <title>''' + dsa_char.name + '''</title>
            <style>
               table {border-collapse: collapse;}
               .td_left {min-width: 200px; width: 200px; background-color: #eeeeee; padding: 2px; border: 1px solid; text-align: right; vertical-align:top; font-weight: bold;}
               .td_right {min-width: 100px; background-color: #ffffff; padding: 2px; border: 1px solid; vertical-align:top;}

               .td_value {min-width: 100px; width: 100px; background-color: #ffffff; padding: 2px; border: 1px solid; vertical-align:top;}
               .td_description {min-width: 250px; background-color: #ffffff; padding: 2px; border: 1px solid; vertical-align:top;}
            </style>
        </head>
        <body>'''

html_content = html_content + '''<h1>{}</h1>'''.format(dsa_char.name)


In [52]:
html_content = html_content + '''\n<h2>{}</h2>\n'''.format("Erfahrungspunkte")
html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
html_content = html_content + '''<tr>'''
html_content = html_content + '''<td class=\"td_left\">Total - verteilt = übrig</td><td class=\"td_right\">{} - {} = <b>{}</b></td>'''.format(dsa_char.experience_total, dsa_char.experience_spent, dsa_char.experience_total - dsa_char.experience_spent)
html_content = html_content + '''</tr>\n'''
html_content = html_content + '''</table>\n'''

html_content = html_content + '''\n<h2>{}</h2>\n'''.format("Status")
html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
html_content = html_content + '''<tr><td class=\"td_left\">Lebenspunkte</td><td class=\"td_right\">{}</td></tr>\n'''.format(dsa_char.life_points)
html_content = html_content + '''<tr><td class=\"td_left\">Astralpunkte</td><td class=\"td_right\">{}</td></tr>\n'''.format(dsa_char.astralenergy)
html_content = html_content + '''<tr><td class=\"td_left\">Schicksalspunkte</td><td class=\"td_right\">{}</td></tr>\n'''.format(dsa_char.fate_points)
html_content = html_content + '''</table>\n'''

html_content = html_content + '''\n<h2>{}</h2>\n'''.format("Details")
html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
for key, value in dsa_char.details.items():
    html_content = html_content + '''<tr>'''
    html_content = html_content + '''<td class=\"td_left\">{}</td><td class=\"td_right\">{}</td>'''.format(key,value)
    html_content = html_content + '''</tr>\n'''
html_content = html_content + '''</table>\n'''


html_content = html_content + '''\n<h2>{}</h2>\n'''.format("Eigenschaften")
html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
for characteristic_key, characteristic_value in dsa_char.characteristics.items():
    html_content = html_content + '''<tr>'''
    html_content = html_content + '''<td class=\"td_left\">{}</td><td class=\"td_right\">{}</td>'''.format(characteristic_key,characteristic_value)
    html_content = html_content + '''</tr>\n'''
html_content = html_content + '''</table>\n'''

for item_type in dsa_char.char_items:
    if len(dsa_char.char_items[item_type]) < 1:
      continue

    html_content = html_content + '''\n<h2>{}</h2>\n'''.format(item_type)

    html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
    for key,value in dsa_char.char_items[item_type].items():
        html_content = html_content + '''<tr>'''
        html_content = html_content + '''<td class=\"td_left\">{}</td><td class=\"td_value\">{}</td><td class=\"td_description\">{}</td>'''.format(key,value.points,value.description)
        html_content = html_content + '''</tr>\n'''

    html_content = html_content + '''</table>\n'''

html_content = html_content + '''</body></html>'''

print(html_content)


<!DOCTYPE html>
        <html>
        <head>
            <title>Hesindian Schlangentreu</title>
            <style>
               table {border-collapse: collapse;}
               .td_left {min-width: 200px; width: 200px; background-color: #eeeeee; padding: 2px; border: 1px solid; text-align: right; vertical-align:top; font-weight: bold;}
               .td_right {min-width: 100px; background-color: #ffffff; padding: 2px; border: 1px solid; vertical-align:top;}

               .td_value {min-width: 100px; width: 100px; background-color: #ffffff; padding: 2px; border: 1px solid; vertical-align:top;}
               .td_description {min-width: 250px; background-color: #ffffff; padding: 2px; border: 1px solid; vertical-align:top;}
            </style>
        </head>
        <body><h1>Hesindian Schlangentreu</h1>
<h2>Erfahrungspunkte</h2>
<table border="1" style="border: 1px solid;">
<tr><td class="td_left">Total - verteilt = übrig</td><td class="td_right">1170 - 1161 = <b>9</b></td></tr

In [53]:
# Convert the generated HTML to PDF
filename = "/content/drive/MyDrive/Colab Notebooks/" + dsa_char.name + ".pdf"
with open(filename, "wb") as pdf_file:
    pisa_status = pisa.CreatePDF(html_content, dest=pdf_file)
    print(pisa_status)
